In [20]:
import pandas as pd
import numpy as np
import json
from together import Together
import os
from dotenv import load_dotenv
from together import Together

# Load API key from .env file
load_dotenv()
API_KEY = os.getenv("TOGETHER_API_KEY")

In [21]:
client = Together(api_key="9e870bfd6d4e11e60c4fb89967d54f32cb7320fca599fe247172bf1f3dc5f898")

high_engagement_posts = []

# Read and collect posts
with open("AskDocs_submissions.csv", "r") as f:
    # Starting with 10,000 rows
    for _ in range(10000):
        line = f.readline()
        if not line:
            break
        post = json.loads(line)

        # Only considering posts that have some engagement
        if post.get("score", 0) > 0:
            high_engagement_posts.append(post)

if not high_engagement_posts:
    print("No high-engagement posts found.")
    exit()

In [9]:
high_engagement_posts.sort(key=lambda x: x.get("score", 0), reverse=True)

# Select top 50% of posts to get posts with the most engagement
top_half_index = len(high_engagement_posts) // 2
top_engagement_posts = high_engagement_posts[:top_half_index]

# Only process 100 posts at first
max_posts = min(len(top_engagement_posts), 100)
top_engagement_posts = top_engagement_posts[:max_posts]

In [22]:
df_selected_posts = pd.DataFrame(top_engagement_posts)
df_selected_posts.to_csv("selected_posts.csv", index=False)

In [10]:
results_prompt1 = []
results_prompt2 = []

for post in top_engagement_posts:
    title = post.get("title", "No title")
    body = post.get("selftext", "No content available")
    score = post.get("score", 0)

    # Prompt 1: Naive Approach - Medical Diagnosis Suggestion
    prompt1 = (
        f"Based on the following symptoms, list possible medical conditions and recommend "
        f"the next diagnostic tests or evaluations a doctor might order. "
        f"Symptoms: {body}. Include brief justifications for each recommendation."
    )

    response1 = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt1}],
    )

    # Prompt 2: Reddit-Style Physician Response
    prompt2 = (
        f"You're replying to a Reddit post in r/AskDocs. The post asks: {body}. "
        f"Write a comment like a real Reddit user who is a verified physician."
    )

    response2 = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": prompt2}],
    )

    common_data = {"title": title, "body": body, "score": score}

    results_prompt1.append({**common_data, "generated_response": response1.choices[0].message.content})
    results_prompt2.append({**common_data, "generated_response": response2.choices[0].message.content})

df_prompt1 = pd.DataFrame(results_prompt1)
df_prompt2 = pd.DataFrame(results_prompt2)

df_prompt1.to_csv("prompt1_responses.csv", index=False)
df_prompt2.to_csv("prompt2_responses.csv", index=False)

print(f"Processed {len(results_prompt1)} posts. Results saved to 'prompt1_responses.csv' and 'prompt2_responses.csv'.")

Processed 100 posts. Results saved to 'prompt1_responses.csv' and 'prompt2_responses.csv'.


In [25]:
results_prompt1[0]['body']

'My 25 year old brother (6\'4", roughly 100 kg, long time IV drug user) was in a motorcycle accident roughly 36 hours ago. He has multiple broken ribs, dislocated hips and a multitude of other internal injuries. He was found at the scene, unconscious with pupils fixed and dilated. In emergency he was declared to have a Glasgow coma scale score of 3. After emergency he had surgery where they removed the top and front of his skull to accommodate the rising intercranial pressure. However, his ICP has been tracking upward steadily from 24 or so up to 90 (not sure what the numbers mean, but I know higher is bad), while his Cerebral Perfusion Pressure (CPP) is down to 25 (from my research, I know this is VERY, VERY bad)... My laymen description is that "he\'s all fucked up". \n\nI would like to know 2 things;\n\n1. What is the worst TBI you have ever seen a patient recover from (I mean survive, discounting brain damage)\n\n2. In this particular case, what are my brother\'s chances of surviva

{'link_flair_css_class': None,
 'gilded': 0,
 'name': 't3_2efzm2',
 'id': '2efzm2',
 'archived': True,
 'author_flair_css_class': None,
 'num_comments': 12,
 'saved': False,
 'over_18': False,
 'is_self': True,
 'from': None,
 'author': '[deleted]',
 'stickied': False,
 'from_id': None,
 'secure_media': None,
 'url': 'http://www.reddit.com/r/AskDocs/comments/2efzm2/doctors_of_reddit_i_only_want_straight_answers/',
 'subreddit_id': 't5_2xtuc',
 'created': 1408892295,
 'from_kind': None,
 'score': 33,
 'edited': 1408975029,
 'quarantine': False,
 'domain': 'self.AskDocs',
 'hide_score': False,
 'created_utc': '1408888695',
 'author_flair_text': None,
 'media_embed': {},
 'subreddit': 'AskDocs',
 'title': 'Doctors of Reddit: I only want straight answers',
 'secure_media_embed': {},
 'downs': 0,
 'thumbnail': 'self',
 'distinguished': None,
 'link_flair_text': None,
 'permalink': '/r/AskDocs/comments/2efzm2/doctors_of_reddit_i_only_want_straight_answers/',
 'ups': 33,
 'retrieved_on': 1441

,link_flair_css_class,gilded,name,id,archived,author_flair_css_class,num_comments,saved,over_18,is_self,...,permalink,ups,retrieved_on,selftext,media,selftext_html,user_reports,mod_reports,banned_by,report_reasons
0,None,0,t3_2efzm2,2efzm2,True,None,12,False,False,True,...,/r/AskDocs/comments/2efzm2/doctors_of_reddit_i...,33,1441204390,"My 25 year old brother (6'4"", roughly 100 kg, ...",None,NaN,NaN,NaN,NaN,NaN
1,None,0,t3_23u7jp,23u7jp,True,verified-doc,11,False,False,True,...,/r/AskDocs/comments/23u7jp/raskdocs_has_reache...,30,1441566554,"/r/AskDocs has reached the 1,000 reader mark a...",None,NaN,NaN,NaN,NaN,NaN
2,None,0,t3_27k4sw,27k4sw,True,None,11,False,False,True,...,/r/AskDocs/comments/27k4sw/my_wife_is_delusion...,21,1441321923,"Dear Reddit,\n\n\nMy wife came home late from ...",None,NaN,NaN,NaN,NaN,NaN
3,None,0,t3_1xqkls,1xqkls,True,verified-doc,23,False,False,True,...,/r/AskDocs/comments/1xqkls/join_the_raskdocs_t...,20,1441916409,"We are currently a small subreddit, but we wou...",None,NaN,NaN,NaN,NaN,NaN
4,None,0,t3_2cwqpo,2cwqpo,True,verified-doc,16,False,False,True,...,/r/AskDocs/comments/2cwqpo/raskdocs_has_merged...,20,1441230685,We are happy to announce that as of today /r/A...,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,None,0,t3_2cqjob,2cqjob,True,None,2,False,True,True,...,/r/AskDocs/comments/2cqjob/should_i_be_worried...,7,1441233637,"Hi, I'm a 20y/o female (obviously) about 5'5"",...",None,NaN,NaN,NaN,NaN,NaN
96,None,0,t3_2d0qbk,2d0qbk,True,None,10,False,False,True,...,/r/AskDocs/comments/2d0qbk/why_does_my_heart_r...,7,1441228825,When I run my HR is about 160bpm for 30-60 min...,None,NaN,NaN,NaN,NaN,NaN
97,None,0,t3_2d2bxp,2d2bxp,True,None,4,False,False,True,...,/r/AskDocs/comments/2d2bxp/i_think_i_had_a_min...,7,1441228018,My nose started to burn then my lip started to...,None,NaN,NaN,NaN,NaN,NaN
98,None,0,t3_2d6q35,2d6q35,True,None,5,False,False,True,...,/r/AskDocs/comments/2d6q35/can_being_in_pain_l...,7,1441225968,"I have a torn peroneal tendon, and have been w...",None,NaN,NaN,NaN,NaN,NaN
